## 기본 환경 설정

In [ ]:
!python --version

In [ ]:
!wget https://www.python.org/ftp/python/3.6.9/Python-3.6.9.tgz
!tar xvfz Python-3.6.9.tgz
!Python-3.6.9/configure
!make
!sudo make install

In [ ]:
!python --version

In [ ]:
!pip install mxnet

In [ ]:
!pip install gluonnlp pandas tqdm

In [ ]:
!pip install sentencepiece

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
!pip install pandas

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [123]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

- torch, nn, F, optim : PyTorch 라이브러리에서 제공되는 모듈, 함수
- Dataset : 데이터 저장, DataLoader : 데이터 관리 및 공급
- gluonnlp : GlunoNLP, 자연어 처리 라이브러리
- numpy : NumPy 라이브러리, 다차원 배열과 행렬 연산에 이용
- tqdm : 진행 상황을 시각적으로 보여주는 라이브러리
- pandas : 데이터 조작 및 분석 라이브러리

In [124]:
# KoBERT
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

- kobert_tokenizer : 한국어 텍스트를 위한 토크나이저
- BertModel : 구글에서 개발된 BERT 모델, 자연어 처리 모델 라이브러리
- AdamW : [Adam 알고리즘 + 가중치 감쇠] 모델의 복잡도를 줄임
- get_cosine_schedule_with_warmup : 학습률을 조절하는 스케줄러

In [125]:
#GPU 사용
device = torch.device("cuda:0")

- GPU : CPU 보다 빠르게 연산되는 병렬 프로세서
- cuda:0 : 첫 번째 GPU를 의미

In [126]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


- tokenizer : 문장을 토큰으로 분리하여, 인덱스로 변환
- bertmodel : 텍스트의 의미 이해, 특성 추출
- vocal : KoBERT의 Vocabulary 설정, 인덱스 매핑

## 데이터셋 전처리

In [133]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [134]:
dataset = pd.read_excel('/content/drive/MyDrive/dataset/dataset.xlsx')

In [135]:
dataset.sample(n=1)

,문장,감정
14179,쓸어다 갖다 다 버려,분노


In [136]:
len(dataset)

96865

In [137]:
dataset["감정"].value_counts()

슬픔    25545
분노    16082
공포    15901
놀람    15702
행복    13376
혐오     5429
중립     4830
Name: 감정, dtype: int64

In [138]:
# 정수 변환
# 슬픔(0), 분노(1), 공포(2), 놀람(3), 행복(4), 혐오(5), 중립(6)
dataset.loc[(dataset["감정"] == "슬픔"), "감정"] = 0
dataset.loc[(dataset["감정"] == "분노"), "감정"] = 1
dataset.loc[(dataset["감정"] == "공포"), "감정"] = 2
dataset.loc[(dataset["감정"] == "놀람"), "감정"] = 3
dataset.loc[(dataset["감정"] == "행복"), "감정"] = 4
dataset.loc[(dataset["감정"] == "혐오"), "감정"] = 5
dataset.loc[(dataset["감정"] == "중립"), "감정"] = 6

In [139]:
dataset["감정"].value_counts()

0    25545
1    16082
2    15901
3    15702
4    13376
5     5429
6     4830
Name: 감정, dtype: int64

In [140]:
dataset.sample(n=10)

,문장,감정
53970,친구가 고시생이어서 걱정되네. 근데 문제는 곧 없어진다는 고시인 사법고시생이야. 아...,2
61602,아내가 아파서 다니던 회사도 그만두고 간호 중인데 점점 우울해지는 것 같아. 응. ...,3
24604,사람들이 똑똑해지기 시작했다 ..,6
73803,생계를 꾸리기 힘든데 연락이 오랫동안 끊긴 딸이 있다는 이유로 지원을 못 받아서 억...,0
62279,요즘 자꾸 우울한데 그 이유를 잘 모르겠어. 일을 하다가도 밥을 먹다가도 갑자기 우...,2
55267,전화로 금융사기를 당해서 모아뒀던 오백만원을 잃었어. 화가 나서 말도 안 나오더라....,1
30708,요리를 더 쉽게 알려 주어서어렵게만 느껴져서 시작도 안 했던 시기를 벗어나요리 초보...,4
88881,정말 죽고 싶다. 왕따를 당했어. 내가 못나서 그런 거 같아서 괴롭다. 이래서 나중...,1
65763,아내가 수술 후 입맛이 없어 힘들어하고 있는데 오늘 내가 만든 보양식으로 회복될 거...,4
34312,쪽발아.쪽발아.,5


In [141]:
data_list = []
for q, label in zip(dataset['문장'], dataset['감정'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)

['친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네. 맞아. 하지만 그렇다고 아무나하고 결혼할 수도 없잖아.', '3']


In [142]:
# train & test 데이터로 분류하기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size= 0.2, random_state = 0)

- test_size= 0.2 : 80%는 train, 20%는 test로 추출
- shuffle = True : 디폴트값으로 생략 가능
- random_state : 난수의 초기값 설정

In [143]:
len(dataset_train)

77492

In [144]:
len(dataset_test)

19373

### 입력 데이터 생성 (토큰화, 정수 인코딩, 패딩)

KoBERT 입력 데이터를 생성하기 위해 다음을 진행해야 한다.


토큰화 : 텍스트를 작은 단위로 나누는 과정

- "안녕하세요" : "안녕", "하세", "요"


정수 인코딩 : 토큰을 고유한 정수로 변환하는 과정

- "안녕" : 1, "하세" : 2, "요" : 3


패딩 : 지정된 입력 크기보다 짧다면, 토큰을 추가하는 과정
- 입력 크기가 10, 입력 문장은 7개의 토큰이라면 3개의 패딩 토큰을 추가하여 10개의 토큰으로 만든다.


In [145]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

- dataset : 원본 데이터셋
- sent_idx : 문장(sentence)의 인덱스
- label_idx : 라벨의 인덱스
- bert_tokenizer : BERT 토크나이저
- vocab : Vocaburary 설정
- max_len : 문장의 최대 길이
- pad : 패딩의 여부
- pair : 문장 쌍 여부

In [149]:
# 파라미터 설정
max_len = 256
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [147]:
#토큰화
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [150]:
tok = tokenizer.tokenize

# BERTDataset(데이터셋, 문장 인덱스, 라벨 인덱스, 토크나이저, 어휘, 최대 길이, 패딩 여부, 문자쌍 여부)
# 토큰화, 정수 인코딩, 패딩 과정 진행
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [151]:
data_train[0]

(array([   2, 3238, 6410, 4627, 5330, 1370, 6897,  517, 6756, 6756, 6166,
        5400, 1168, 6855,   54, 1435, 2110, 3943, 7088, 4955,  881, 3093,
        5777, 5591, 2497, 7788,  880, 5876,   54, 1435,  517, 6249, 3943,
        7868, 7261, 2050, 5439,   54, 3945, 5859, 3942, 3647, 6099, 2872,
        3278,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [152]:
data_test[0]

(array([   2, 1435,  891, 3177, 7086, 2397, 7418, 5330, 3135, 1771, 5439,
        5007, 4619, 6441, 4012, 6398, 6116, 2426, 7852, 4998,   54, 1458,
        3239, 7788, 2856, 6527, 7848,   54, 4044, 2393, 5889, 3177, 2397,
        6896,  758, 7864, 6887, 1435,  891, 4832, 5859, 6824, 7086, 4004,
        7096, 3135, 1771, 5439, 4924, 5839,   54, 2397, 3475, 4332, 2233,
        2872, 3860, 2270, 7088, 4529, 3175, 6364, 6061, 4998,   54,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

1번 array는 패딩된 시퀀스, 1이라는 패딩 토큰 추가하여 길이 조정

2번 array는 패딩 전 원본 시퀀스의 길이와, 데이터 유형

3번 array는 어텐션 마스크 시퀀스

어텐션 마스크를 사용하여, 입력 시퀀스에서 주의해야 할 토큰과 무시해야 하는 토큰을 나타낸다. 어텐션 마스크는 0으로, 실제 연산에 필요한 토큰의 위치는 1로 채워진다.

In [153]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
